In [2]:
import numpy as np
import pandas as pd
import requests

In [3]:
df_trips = pd.read_csv('trips_1.5M.csv')
df_trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount
0,1,2015-06-03 10:41:50,2015-06-03 11:08:33,1,2.80,-73.952888,40.776814,1,N,-73.979034,40.756611,1,17.5,0.0,0.5,4.57,0.0,22.87
1,2,2015-02-14 19:38:55,2015-02-14 20:00:59,1,2.79,-73.995827,40.725353,1,N,-73.984314,40.757389,2,15.0,0.0,0.5,0.00,0.0,15.80
2,2,2015-04-21 15:21:04,2015-04-21 15:33:38,1,1.75,-73.951920,40.769421,1,N,-73.956421,40.787140,2,9.5,0.0,0.5,0.00,0.0,10.30
3,2,2015-01-29 18:58:54,2015-01-29 19:04:46,3,0.83,-74.009026,40.715710,1,N,-74.003418,40.723152,1,5.5,1.0,0.5,1.46,0.0,8.76
4,1,2015-06-05 10:47:29,2015-06-05 10:58:00,3,0.80,-73.985909,40.756176,1,N,-73.988358,40.747398,2,7.0,0.0,0.5,0.00,0.0,7.80


In [18]:
df_trips.info()
df_trips.columns
df_trips['VendorID'] = df_trips['VendorID'].astype('int8')
df_trips['passenger_count'] = df_trips['passenger_count'].astype('int8')
df_trips['RateCodeID'] = df_trips['RateCodeID'].astype('int8')
df_trips['payment_type'] = df_trips['payment_type'].astype('int8')
df_trips.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500000 entries, 0 to 1499999
Data columns (total 18 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   VendorID               1500000 non-null  int8   
 1   tpep_pickup_datetime   1500000 non-null  object 
 2   tpep_dropoff_datetime  1500000 non-null  object 
 3   passenger_count        1500000 non-null  int8   
 4   trip_distance          1500000 non-null  float64
 5   pickup_longitude       1500000 non-null  float64
 6   pickup_latitude        1500000 non-null  float64
 7   RateCodeID             1500000 non-null  int64  
 8   store_and_fwd_flag     1500000 non-null  object 
 9   dropoff_longitude      1500000 non-null  float64
 10  dropoff_latitude       1500000 non-null  float64
 11  payment_type           1500000 non-null  int64  
 12  fare_amount            1500000 non-null  float16
 13  extra                  1500000 non-null  float64
 14  mta_tax           

In [22]:
df_trips[['fare_amount','dropoff_longitude', 'dropoff_latitude', 'pickup_longitude', 'pickup_latitude', 'extra', 'mta_tax', 'tip_amount','tolls_amount' ]] = df_trips[['fare_amount','dropoff_longitude', 'dropoff_latitude', 'pickup_longitude', 'pickup_latitude', 'extra', 'mta_tax', 'tip_amount','tolls_amount']].astype('float16')
df_trips['total_amount'] = df_trips['total_amount'].astype('float32')


In [23]:
df_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500000 entries, 0 to 1499999
Data columns (total 18 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   VendorID               1500000 non-null  int8   
 1   tpep_pickup_datetime   1500000 non-null  object 
 2   tpep_dropoff_datetime  1500000 non-null  object 
 3   passenger_count        1500000 non-null  int8   
 4   trip_distance          1500000 non-null  float64
 5   pickup_longitude       1500000 non-null  float16
 6   pickup_latitude        1500000 non-null  float16
 7   RateCodeID             1500000 non-null  int8   
 8   store_and_fwd_flag     1500000 non-null  object 
 9   dropoff_longitude      1500000 non-null  float16
 10  dropoff_latitude       1500000 non-null  float16
 11  payment_type           1500000 non-null  int8   
 12  fare_amount            1500000 non-null  float16
 13  extra                  1500000 non-null  float16
 14  mta_tax           

In [ ]:
df_trips